# Discovery of VTK

Date: 7th May 2019

Company: EMBL-EBI

Group: Virginie Uhlmann

Author: Yoann Pradat

Source code: https://vtk.org/Wiki/VTK/Examples/Python

### Import libraries

In [16]:
import numpy as np
import vtk

from vtk import vtkCellArray
from vtk import vtkPoints
from vtk import vtkActor
from vtk import vtkIdTypeArray
from vtk import vtkIntArray
from vtk import vtkPolyData
from vtk import vtkPolyDataMapper
from vtk import vtkSphereSource
import vtk.util.numpy_support as vtk_np

from vtk.util.colors import tomato

### Reconstruct of a surface

In [17]:
def sample_sphere(M_1, M_2):
    """
    This function samples M_1*M_2 points from the continuous representation of a sphere.
    
    Parameters
    ----------
    M_1: int
        Number of points in the first conintuous dimension
    M_2: int 
        Number of points in the second continous dimension
    Return
    ---------
    points: np.array(M_1*M_2, 3)
        Array of 3D points sampled from the sphere
    """
    
    T_1, T_2 = 1/M_1, 1/M_2
    u = np.r_[0:1:T_1].reshape(-1, 1)
    v = np.r_[0:1:T_2].reshape(-1, 1)
    
    x = np.cos(2*np.pi*u).dot(np.sin(np.pi*v).T)
    y = np.sin(2*np.pi*u).dot(np.sin(np.pi*v).T)
    z = np.ones((M_1, 1)).dot(np.cos(np.pi*v).T)
    
    points = np.stack((x.flatten(), y.flatten(), z.flatten()), axis=1)
    return points

# Random topography + spline

In [18]:
#!/usr/bin/env python

import vtk
import random
import numpy


# Make a 32 x 32 grid
size = 32

# Define z values for the topography
topography = numpy.zeros([size, size])
for i in range(size):
    for j in range(size):
        topography[i][j] = random.randrange(0, 5)

# Define points, triangles and colors
colors = vtk.vtkUnsignedCharArray()
colors.SetNumberOfComponents(3)
points = vtk.vtkPoints()
triangles = vtk.vtkCellArray()

# Build the meshgrid manually
count = 0
for i in range(size-1):
    for j in range(size-1):

        z1 = topography[i][j]
        z2 = topography[i][j+1]
        z3 = topography[i+1][j]

        # Triangle 1
        points.InsertNextPoint(i, j, z1)
        points.InsertNextPoint(i, (j+1), z2)
        points.InsertNextPoint((i+1), j, z3)

        triangle = vtk.vtkTriangle()
        triangle.GetPointIds().SetId(0, count)
        triangle.GetPointIds().SetId(1, count + 1)
        triangle.GetPointIds().SetId(2, count + 2)

        triangles.InsertNextCell(triangle)

        z1 = topography[i][j+1]
        z2 = topography[i+1][j+1]
        z3 = topography[i+1][j]

        # Triangle 2
        points.InsertNextPoint(i, (j+1), z1)
        points.InsertNextPoint((i+1), (j+1), z2)
        points.InsertNextPoint((i+1), j, z3)

        triangle = vtk.vtkTriangle()
        triangle.GetPointIds().SetId(0, count + 3)
        triangle.GetPointIds().SetId(1, count + 4)
        triangle.GetPointIds().SetId(2, count + 5)

        count += 6

        triangles.InsertNextCell(triangle)

        # Add some color
        r = [int(i/float(size)*255),int(j/float(size)*255),0]
        colors.InsertNextTuple(r)
        colors.InsertNextTuple(r)
        colors.InsertNextTuple(r)
        colors.InsertNextTuple(r)
        colors.InsertNextTuple(r)
        colors.InsertNextTuple(r)

# Create a polydata object
trianglePolyData = vtk.vtkPolyData()

# Add the geometry and topology to the polydata
trianglePolyData.SetPoints(points)
trianglePolyData.GetPointData().SetScalars(colors)
trianglePolyData.SetPolys(triangles)

# Clean the polydata so that the edges are shared !
cleanPolyData = vtk.vtkCleanPolyData()
cleanPolyData.SetInputData(trianglePolyData)

# Use a filter to smooth the data (will add triangles and smooth)
smooth_loop = vtk.vtkLoopSubdivisionFilter()
smooth_loop.SetNumberOfSubdivisions(3)
smooth_loop.SetInputConnection(cleanPolyData.GetOutputPort())

# Create a mapper and actor for smoothed dataset
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(smooth_loop.GetOutputPort())
actor_loop = vtk.vtkActor()
actor_loop.SetMapper(mapper)
actor_loop.GetProperty().SetInterpolationToFlat()

# Update the pipeline so that vtkCellLocator finds cells !
smooth_loop.Update()

# Define a cellLocator to be able to compute intersections between lines
# and the surface
locator = vtk.vtkCellLocator()
locator.SetDataSet(smooth_loop.GetOutput())
locator.BuildLocator()

maxloop = 1000
dist = 20.0/maxloop
tolerance = 0.001

# Make a list of points. Each point is the intersection of a vertical line
# defined by p1 and p2 and the surface.
points = vtk.vtkPoints()
for i in range(maxloop):

    p1 = [2+i*dist, 16, -1]
    p2 = [2+i*dist, 16, 6]

    # Outputs (we need only pos which is the x, y, z position
    # of the intersection)
    t = vtk.mutable(0)
    pos = [0.0, 0.0, 0.0]
    pcoords = [0.0, 0.0, 0.0]
    subId = vtk.mutable(0)
    locator.IntersectWithLine(p1, p2, tolerance, t, pos, pcoords, subId)

    # Add a slight offset in z
    pos[2] += 0.01
    # Add the x, y, z position of the intersection
    points.InsertNextPoint(pos)

# Create a spline and add the points
spline = vtk.vtkParametricSpline()
spline.SetPoints(points)
functionSource = vtk.vtkParametricFunctionSource()
functionSource.SetUResolution(maxloop)
functionSource.SetParametricFunction(spline)

# Map the spline
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(functionSource.GetOutputPort())

# Define the line actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor([1.0, 0.0, 0.0])
actor.GetProperty().SetLineWidth(3)

# Visualize
renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)

# Add actors and render
renderer.AddActor(actor)
renderer.AddActor(actor_loop)

renderer.SetBackground(1, 1, 1)  # Background color white
renderWindow.SetSize(800, 800)
renderWindow.Render()
renderWindowInteractor.Start()

# Surface from Point Clouds Sphere + Cactus

## Sphere

In [3]:
# This example shows how to construct a surface from a point cloud.
# First we generate a volume using the
# vtkSurfaceReconstructionFilter. The volume values are a distance
# field. Once this is generated, the volume is countoured at a
# distance value of 0.0.


# Read some points. Use a programmable filter to read them.
pointSource = vtk.vtkProgrammableSource()

M_1, M_2 = 50, 50

def readPoints():
    output = pointSource.GetPolyDataOutput()
    points = vtk.vtkPoints()
    output.SetPoints(points)

    pointSphere = sample_sphere(M_1, M_2)
    for x, y, z in pointSphere:
        points.InsertNextPoint(x, y, z)

pointSource.SetExecuteMethod(readPoints)


# Construct the surface and create isosurface.
surf = vtk.vtkSurfaceReconstructionFilter()
surf.SetInputConnection(pointSource.GetOutputPort())

cf = vtk.vtkContourFilter()
cf.SetInputConnection(surf.GetOutputPort())
cf.SetValue(0, 0.0)

# Sometimes the contouring algorithm can create a volume whose gradient
# vector and ordering of polygon (using the right hand rule) are
# inconsistent. vtkReverseSense cures this problem.
reverse = vtk.vtkReverseSense()
reverse.SetInputConnection(cf.GetOutputPort())
reverse.ReverseCellsOn()
reverse.ReverseNormalsOn()

map = vtk.vtkPolyDataMapper()
map.SetInputConnection(reverse.GetOutputPort())
map.ScalarVisibilityOff()

surfaceActor = vtk.vtkActor()
surfaceActor.SetMapper(map)
surfaceActor.GetProperty().SetDiffuseColor(1.0000, 0.3882, 0.2784)
surfaceActor.GetProperty().SetSpecularColor(1, 1, 1)
surfaceActor.GetProperty().SetSpecular(.4)
surfaceActor.GetProperty().SetSpecularPower(50)

# Create the RenderWindow, Renderer and both Actors
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# Add the actors to the renderer, set the background and size
ren.AddActor(surfaceActor)
ren.SetBackground(1, 1, 1)
renWin.SetSize(400, 400)
ren.GetActiveCamera().SetFocalPoint(0, 0, 0)
ren.GetActiveCamera().SetPosition(1, 0, 0)
ren.GetActiveCamera().SetViewUp(0, 0, 1)
ren.ResetCamera()
ren.GetActiveCamera().Azimuth(20)
ren.GetActiveCamera().Elevation(30)
ren.GetActiveCamera().Dolly(1.2)
ren.ResetCameraClippingRange()

iren.Initialize()
renWin.Render()
iren.Start()

## Cactus

In [19]:
# Read some points. Use a programmable filter to read them.
pointSource = vtk.vtkProgrammableSource()

def readPoints():
    output = pointSource.GetPolyDataOutput()
    points = vtk.vtkPoints()
    output.SetPoints(points)

    file = open("../data/cactus.3337.pts")

    line = file.readline()
    while line:
        data = line.split()
        if data and data[0] == 'p':
            x, y, z = float(data[1]), float(data[2]), float(data[3])
            points.InsertNextPoint(x, y, z)
        line = file.readline()

pointSource.SetExecuteMethod(readPoints)


# Construct the surface and create isosurface.
surf = vtk.vtkSurfaceReconstructionFilter()
surf.SetInputConnection(pointSource.GetOutputPort())

cf = vtk.vtkContourFilter()
cf.SetInputConnection(surf.GetOutputPort())
cf.SetValue(0, 0.0)

# Sometimes the contouring algorithm can create a volume whose gradient
# vector and ordering of polygon (using the right hand rule) are
# inconsistent. vtkReverseSense cures this problem.
reverse = vtk.vtkReverseSense()
reverse.SetInputConnection(cf.GetOutputPort())
reverse.ReverseCellsOn()
reverse.ReverseNormalsOn()

map = vtk.vtkPolyDataMapper()
map.SetInputConnection(reverse.GetOutputPort())
map.ScalarVisibilityOff()

surfaceActor = vtk.vtkActor()
surfaceActor.SetMapper(map)
surfaceActor.GetProperty().SetDiffuseColor(1.0000, 0.3882, 0.2784)
surfaceActor.GetProperty().SetSpecularColor(1, 1, 1)
surfaceActor.GetProperty().SetSpecular(.4)
surfaceActor.GetProperty().SetSpecularPower(50)

# Create the RenderWindow, Renderer and both Actors
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# Add the actors to the renderer, set the background and size
ren.AddActor(surfaceActor)
ren.SetBackground(1, 1, 1)
renWin.SetSize(400, 400)
ren.GetActiveCamera().SetFocalPoint(0, 0, 0)
ren.GetActiveCamera().SetPosition(1, 0, 0)
ren.GetActiveCamera().SetViewUp(0, 0, 1)
ren.ResetCamera()
ren.GetActiveCamera().Azimuth(20)
ren.GetActiveCamera().Elevation(30)
ren.GetActiveCamera().Dolly(1.2)
ren.ResetCameraClippingRange()

iren.Initialize()
renWin.Render()
iren.Start()

# Sphere from VTK

In [20]:
sphere = vtk.vtkSphere()
sphere.SetCenter(0, 0, 0)
sphere.SetRadius(0.5)

# The sample function generates a distance function from the implicit
# function. This is then contoured to get a polygonal surface.
sample = vtk.vtkSampleFunction()
sample.SetImplicitFunction(sphere)
sample.SetModelBounds(-.5, .5, -.5, .5, -.5, .5)
sample.SetSampleDimensions(50, 50, 50)
sample.ComputeNormalsOff()

# contour
surface = vtk.vtkContourFilter()
surface.GenerateTrianglesOn()
surface.SetInputConnection(sample.GetOutputPort())
surface.SetValue(0, 0.0)

# mapper
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(surface.GetOutputPort())
mapper.ScalarVisibilityOff()
actor = vtk.vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().EdgeVisibilityOn()
actor.GetProperty().SetEdgeColor(.2, .2, .5)

# A renderer and render window
renderer = vtk.vtkRenderer()
renderer.SetBackground(1, 1, 1)

# add the actor
renderer.AddActor(actor)

# render window
renwin = vtk.vtkRenderWindow()
renwin.AddRenderer(renderer)
renwin.SetSize(900, 900)

# An interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(renwin)

# Start
interactor.Initialize()
interactor.Start()

# 3D Objects

In [21]:
def MakeHexagonalPrism():
    """
      3D: hexagonal prism: a wedge with an hexagonal base.
      Be careful, the base face ordering is different from wedge.
    """

    numberOfVertices = 12

    points = vtk.vtkPoints()

    points.InsertNextPoint(0.0, 0.0, 1.0)
    points.InsertNextPoint(1.0, 0.0, 1.0)
    points.InsertNextPoint(1.5, 0.5, 1.0)
    points.InsertNextPoint(1.0, 1.0, 1.0)
    points.InsertNextPoint(0.0, 1.0, 1.0)
    points.InsertNextPoint(-0.5, 0.5, 1.0)

    points.InsertNextPoint(0.0, 0.0, 0.0)
    points.InsertNextPoint(1.0, 0.0, 0.0)
    points.InsertNextPoint(1.5, 0.5, 0.0)
    points.InsertNextPoint(1.0, 1.0, 0.0)
    points.InsertNextPoint(0.0, 1.0, 0.0)
    points.InsertNextPoint(-0.5, 0.5, 0.0)

    hexagonalPrism = vtk.vtkHexagonalPrism()
    for i in range(0, numberOfVertices):
        hexagonalPrism.GetPointIds().SetId(i, i)

    ug = vtk.vtkUnstructuredGrid()
    ug.InsertNextCell(hexagonalPrism.GetCellType(),
                      hexagonalPrism.GetPointIds())
    ug.SetPoints(points)

    return ug


def MakeHexahedron():
    """
      A regular hexagon (cube) with all faces square and three squares around
       each vertex is created below.

      Setup the coordinates of eight points
       (the two faces must be in counter clockwise
       order as viewed from the outside).

      As an exercise you can modify the coordinates of the points to create
       seven topologically distinct convex hexahedras.
    """
    numberOfVertices = 8

    # Create the points
    points = vtk.vtkPoints()
    points.InsertNextPoint(0.0, 0.0, 0.0)
    points.InsertNextPoint(1.0, 0.0, 0.0)
    points.InsertNextPoint(1.0, 1.0, 0.0)
    points.InsertNextPoint(0.0, 1.0, 0.0)
    points.InsertNextPoint(0.0, 0.0, 1.0)
    points.InsertNextPoint(1.0, 0.0, 1.0)
    points.InsertNextPoint(1.0, 1.0, 1.0)
    points.InsertNextPoint(0.0, 1.0, 1.0)

    # Create a hexahedron from the points
    hex_ = vtk.vtkHexahedron()
    for i in range(0, numberOfVertices):
        hex_.GetPointIds().SetId(i, i)

    # Add the points and hexahedron to an unstructured grid
    uGrid = vtk.vtkUnstructuredGrid()
    uGrid.SetPoints(points)
    uGrid.InsertNextCell(hex_.GetCellType(), hex_.GetPointIds())

    return uGrid


def MakePentagonalPrism():
    numberOfVertices = 10

    # Create the points
    points = vtk.vtkPoints()
    points.InsertNextPoint(11, 10, 10)
    points.InsertNextPoint(13, 10, 10)
    points.InsertNextPoint(14, 12, 10)
    points.InsertNextPoint(12, 14, 10)
    points.InsertNextPoint(10, 12, 10)
    points.InsertNextPoint(11, 10, 14)
    points.InsertNextPoint(13, 10, 14)
    points.InsertNextPoint(14, 12, 14)
    points.InsertNextPoint(12, 14, 14)
    points.InsertNextPoint(10, 12, 14)

    # Pentagonal Prism
    pentagonalPrism = vtk.vtkPentagonalPrism()
    for i in range(0, numberOfVertices):
        pentagonalPrism.GetPointIds().SetId(i, i)

    # Add the points and hexahedron to an unstructured grid
    uGrid = vtk.vtkUnstructuredGrid()
    uGrid.SetPoints(points)
    uGrid.InsertNextCell(pentagonalPrism.GetCellType(),
                         pentagonalPrism.GetPointIds())

    return uGrid


def MakePolyhedron():
    """
      Make a regular dodecahedron. It consists of twelve regular pentagonal
      faces with three faces meeting at each vertex.
    """
    # numberOfVertices = 20
    numberOfFaces = 12
    # numberOfFaceVertices = 5

    points = vtk.vtkPoints()
    points.InsertNextPoint(1.21412, 0, 1.58931)
    points.InsertNextPoint(0.375185, 1.1547, 1.58931)
    points.InsertNextPoint(-0.982247, 0.713644, 1.58931)
    points.InsertNextPoint(-0.982247, -0.713644, 1.58931)
    points.InsertNextPoint(0.375185, -1.1547, 1.58931)
    points.InsertNextPoint(1.96449, 0, 0.375185)
    points.InsertNextPoint(0.607062, 1.86835, 0.375185)
    points.InsertNextPoint(-1.58931, 1.1547, 0.375185)
    points.InsertNextPoint(-1.58931, -1.1547, 0.375185)
    points.InsertNextPoint(0.607062, -1.86835, 0.375185)
    points.InsertNextPoint(1.58931, 1.1547, -0.375185)
    points.InsertNextPoint(-0.607062, 1.86835, -0.375185)
    points.InsertNextPoint(-1.96449, 0, -0.375185)
    points.InsertNextPoint(-0.607062, -1.86835, -0.375185)
    points.InsertNextPoint(1.58931, -1.1547, -0.375185)
    points.InsertNextPoint(0.982247, 0.713644, -1.58931)
    points.InsertNextPoint(-0.375185, 1.1547, -1.58931)
    points.InsertNextPoint(-1.21412, 0, -1.58931)
    points.InsertNextPoint(-0.375185, -1.1547, -1.58931)
    points.InsertNextPoint(0.982247, -0.713644, -1.58931)

    # Dimensions are [numberOfFaces][numberOfFaceVertices]
    dodechedronFace = [
        [0, 1, 2, 3, 4],
        [0, 5, 10, 6, 1],
        [1, 6, 11, 7, 2],
        [2, 7, 12, 8, 3],
        [3, 8, 13, 9, 4],
        [4, 9, 14, 5, 0],
        [15, 10, 5, 14, 19],
        [16, 11, 6, 10, 15],
        [17, 12, 7, 11, 16],
        [18, 13, 8, 12, 17],
        [19, 14, 9, 13, 18],
        [19, 18, 17, 16, 15]
    ]

    dodechedronFacesIdList = vtk.vtkIdList()
    # Number faces that make up the cell.
    dodechedronFacesIdList.InsertNextId(numberOfFaces)
    for face in dodechedronFace:
        # Number of points in the face == numberOfFaceVertices
        dodechedronFacesIdList.InsertNextId(len(face))
        # Insert the pointIds for that face.
        [dodechedronFacesIdList.InsertNextId(i) for i in face]

    uGrid = vtk.vtkUnstructuredGrid()
    uGrid.InsertNextCell(vtk.VTK_POLYHEDRON, dodechedronFacesIdList)
    uGrid.SetPoints(points)

    return uGrid


def MakePyramid():
    """
      Make a regular square pyramid.
    """
    numberOfVertices = 5

    points = vtk.vtkPoints()

    p = [
        [1.0, 1.0, 0.0],
        [-1.0, 1.0, 0.0],
        [-1.0, -1.0, 0.0],
        [1.0, -1.0, 0.0],
        [0.0, 0.0, 1.0]
    ]
    for pt in p:
        points.InsertNextPoint(pt)

    pyramid = vtk.vtkPyramid()
    for i in range(0, numberOfVertices):
        pyramid.GetPointIds().SetId(i, i)

    ug = vtk.vtkUnstructuredGrid()
    ug.SetPoints(points)
    ug.InsertNextCell(pyramid.GetCellType(), pyramid.GetPointIds())

    return ug


def MakeTetrahedron():
    """
      Make a tetrahedron.
    """
    numberOfVertices = 4

    points = vtk.vtkPoints()
    points.InsertNextPoint(0, 0, 0)
    points.InsertNextPoint(1, 0, 0)
    points.InsertNextPoint(1, 1, 0)
    points.InsertNextPoint(0, 1, 1)

    tetra = vtk.vtkTetra()
    for i in range(0, numberOfVertices):
        tetra.GetPointIds().SetId(i, i)

    cellArray = vtk.vtkCellArray()
    cellArray.InsertNextCell(tetra)

    unstructuredGrid = vtk.vtkUnstructuredGrid()
    unstructuredGrid.SetPoints(points)
    unstructuredGrid.SetCells(vtk.VTK_TETRA, cellArray)

    return unstructuredGrid


def MakeVoxel():
    """
      A voxel is a representation of a regular grid in 3-D space.
    """
    numberOfVertices = 8

    points = vtk.vtkPoints()
    points.InsertNextPoint(0, 0, 0)
    points.InsertNextPoint(1, 0, 0)
    points.InsertNextPoint(0, 1, 0)
    points.InsertNextPoint(1, 1, 0)
    points.InsertNextPoint(0, 0, 1)
    points.InsertNextPoint(1, 0, 1)
    points.InsertNextPoint(0, 1, 1)
    points.InsertNextPoint(1, 1, 1)

    voxel = vtk.vtkVoxel()
    for i in range(0, numberOfVertices):
        voxel.GetPointIds().SetId(i, i)

    ug = vtk.vtkUnstructuredGrid()
    ug.SetPoints(points)
    ug.InsertNextCell(voxel.GetCellType(), voxel.GetPointIds())

    return ug


def MakeWedge():
    """
      A wedge consists of two triangular ends and three rectangular faces.
    """

    numberOfVertices = 6

    points = vtk.vtkPoints()

    points.InsertNextPoint(0, 1, 0)
    points.InsertNextPoint(0, 0, 0)
    points.InsertNextPoint(0, .5, .5)
    points.InsertNextPoint(1, 1, 0)
    points.InsertNextPoint(1, 0.0, 0.0)
    points.InsertNextPoint(1, .5, .5)

    wedge = vtk.vtkWedge()
    for i in range(0, numberOfVertices):
        wedge.GetPointIds().SetId(i, i)

    ug = vtk.vtkUnstructuredGrid()
    ug.SetPoints(points)
    ug.InsertNextCell(wedge.GetCellType(), wedge.GetPointIds())

    return ug


def WritePNG(renWin, fn, magnification=1):
    """
      Screenshot

      Write out a png corresponding to the render window.

      :param: renWin - the render window.
      :param: fn - the file name.
      :param: magnification - the magnification.
    """
    windowToImageFilter = vtk.vtkWindowToImageFilter()
    windowToImageFilter.SetInput(renWin)
    windowToImageFilter.SetMagnification(magnification)
    # Record the alpha (transparency) channel
    # windowToImageFilter.SetInputBufferTypeToRGBA()
    windowToImageFilter.SetInputBufferTypeToRGB()
    # Read from the back buffer
    windowToImageFilter.ReadFrontBufferOff()
    windowToImageFilter.Update()

    writer = vtk.vtkPNGWriter()
    writer.SetFileName(fn)
    writer.SetInputConnection(windowToImageFilter.GetOutputPort())
    writer.Write()

In [22]:
colors = vtk.vtkNamedColors()

# Set the background color.
colors.SetColor("BkgColor", [51, 77, 102, 255])

titles = list()
textMappers = list()
textActors = list()

uGrids = list()
mappers = list()
actors = list()
renderers = list()

uGrids.append(MakeHexagonalPrism())
titles.append('Hexagonal Prism')
uGrids.append(MakeHexahedron())
titles.append('Hexahedron')
uGrids.append(MakePentagonalPrism())
titles.append('Pentagonal Prism')

uGrids.append(MakePolyhedron())
titles.append('Polyhedron')
uGrids.append(MakePyramid())
titles.append('Pyramid')
uGrids.append(MakeTetrahedron())
titles.append('Tetrahedron')

uGrids.append(MakeVoxel())
titles.append('Voxel')
uGrids.append(MakeWedge())
titles.append('Wedge')

renWin = vtk.vtkRenderWindow()
renWin.SetWindowName('Cell3D Demonstration')

iRen = vtk.vtkRenderWindowInteractor()
iRen.SetRenderWindow(renWin)

# Create one text property for all
textProperty = vtk.vtkTextProperty()
textProperty.SetFontSize(16)
textProperty.SetJustificationToCentered()

# Create and link the mappers actors and renderers together.
for i in range(0, len(uGrids)):
    textMappers.append(vtk.vtkTextMapper())
    textActors.append(vtk.vtkActor2D())

    mappers.append(vtk.vtkDataSetMapper())
    actors.append(vtk.vtkActor())
    renderers.append(vtk.vtkRenderer())

    mappers[i].SetInputData(uGrids[i])
    actors[i].SetMapper(mappers[i])
    actors[i].GetProperty().SetColor(
        colors.GetColor3d("Seashell"))
    renderers[i].AddViewProp(actors[i])

    textMappers[i].SetInput(titles[i])
    textMappers[i].SetTextProperty(textProperty)

    textActors[i].SetMapper(textMappers[i])
    textActors[i].SetPosition(120, 16)
    renderers[i].AddViewProp(textActors[i])

    renWin.AddRenderer(renderers[i])

gridDimensions = 3
rendererSize = 300

renWin.SetSize(rendererSize * gridDimensions,
               rendererSize * gridDimensions)

for row in range(0, gridDimensions):
    for col in range(0, gridDimensions):
        index = row * gridDimensions + col

        # (xmin, ymin, xmax, ymax)
        viewport = [
            float(col) * rendererSize /
            (gridDimensions * rendererSize),
            float(gridDimensions - (row + 1)) * rendererSize /
            (gridDimensions * rendererSize),
            float(col + 1) * rendererSize /
            (gridDimensions * rendererSize),
            float(gridDimensions - row) * rendererSize /
            (gridDimensions * rendererSize)]

        if index > len(actors) - 1:
            # Add a renderer even if there is no actor.
            # This makes the render window background all the same color.
            ren = vtk.vtkRenderer()
            ren.SetBackground(colors.GetColor3d("BkgColor"))
            ren.SetViewport(viewport)
            renWin.AddRenderer(ren)
            continue

        renderers[index].SetViewport(viewport)
        renderers[index].SetBackground(colors.GetColor3d("BkgColor"))
        renderers[index].ResetCamera()
        renderers[index].GetActiveCamera().Azimuth(30)
        renderers[index].GetActiveCamera().Elevation(-30)
        renderers[index].GetActiveCamera().Zoom(0.85)
        renderers[index].ResetCameraClippingRange()

iRen.Initialize()
renWin.Render()
iRen.Start()

In [8]:
help(scaleData)

Help on vtkPolyData object:

class vtkPolyData(vtkPointSet)
 |  vtkPolyData - concrete dataset represents vertices, lines, polygons,
 |  and triangle strips
 |  
 |  Superclass: vtkPointSet
 |  
 |  vtkPolyData is a data object that is a concrete implementation of
 |  vtkDataSet. vtkPolyData represents a geometric structure consisting
 |  of vertices, lines, polygons, and/or triangle strips. Point and cell
 |  attribute values (e.g., scalars, vectors, etc.) also are represented.
 |  
 |  The actual cell types (vtkCellType.h) supported by vtkPolyData are:
 |  vtkVertex, vtkPolyVertex, vtkLine, vtkPolyLine, vtkTriangle, vtkQuad,
 |  vtkPolygon, and vtkTriangleStrip.
 |  
 |  One important feature of vtkPolyData objects is that special
 |  traversal and data manipulation methods are available to process
 |  data. These methods are generally more efficient than vtkDataSet
 |  methods and should be used whenever possible. For example, traversing
 |  the cells in a dataset we would use GetCe